<a href="https://colab.research.google.com/github/hmg8906/CienciaDatosCN/blob/main/DefoRisk_modelo_predictivo_de_riesgo_de_deforestaci%C3%B3n_para_AOI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
#inicializar y autenticar GEE
import ee
ee.Authenticate()
ee.Initialize(project="hmgarcia56ee1")

In [48]:
import ee
import geemap
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

def calculate_deforestation_risk(aoi, num_trees=100, num_samples=2000, export_resolution=20):
    """
    Calcula el riesgo de deforestación usando Random Forest en Google Earth Engine.

    Parámetros:
    aoi: ee.FeatureCollection - Área de interés
    num_trees: int - Número de árboles en el modelo Random Forest
    num_samples: int - Número de puntos de muestreo aleatorios
    export_resolution: int - Resolución en metros para la exportación

    Retorna:
    geemap.Map - Mapa interactivo con las capas de riesgo de deforestación
    """
    try:
        # Cargar capas de entrada
        forest_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
        forest_loss = forest_data.select("loss")  # Pérdida de bosque (binaria: 0 o 1)
        tree_cover = forest_data.select("treecover2000")  # Cobertura arbórea en el año 2000
        elevation = ee.Image("USGS/SRTMGL1_003")  # Modelo de elevación digital
        slope = ee.Terrain.slope(elevation)  # Pendiente derivada de la elevación
        roads = ee.FeatureCollection("TIGER/2016/Roads")  # Red de carreteras

        # Convertir carreteras a ráster y calcular distancia a carreteras
        roads = roads.map(lambda f: f.set("count", 1))  # Asegurar propiedad numérica
        roads_raster = roads.reduceToImage(properties=["count"], reducer=ee.Reducer.sum()).gt(0).rename("road_presence")
        road_distance = roads_raster.fastDistanceTransform(30).rename("road_distance")  # Distancia a carreteras

        # Cargar variables edáficas (seleccionar solo la banda de interés)
        soil_texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").select("b0").rename("soil_texture")
        soil_carbon = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select("b0").rename("soil_carbon")

        # Preparar dataset de entrenamiento (sin densidad de población)
        training_bands = [tree_cover, elevation, slope, road_distance, soil_texture, soil_carbon]
        training_stack = ee.Image.cat(training_bands).float()  # Convertir a tipo flotante

        # Generar puntos de muestreo aleatorios
        loss_mask = forest_loss.gt(0)  # Máscara de píxeles con pérdida de bosque
        points = loss_mask.addBands(training_stack).sample(
            region=aoi,
            scale=30,
            numPixels=num_samples * 5,  # Muestrear más puntos para asegurar suficientes datos
            seed=42,
            geometries=True
        ).filter(ee.Filter.notNull(training_stack.bandNames()))  # Filtrar puntos nulos

        # Validar si hay suficientes datos
        count = points.size().getInfo()
        print(f"Número de puntos de muestreo generados: {count}")
        if count == 0:
            raise ValueError("No se encontraron suficientes datos de entrenamiento.")

        # Dividir en conjuntos de entrenamiento y prueba
        training_fc = points.randomColumn()
        training_set = training_fc.filter(ee.Filter.lt('random', 0.8))
        testing_set = training_fc.filter(ee.Filter.gte('random', 0.8))

        # Entrenar modelo Random Forest en GEE
        classifier = ee.Classifier.smileRandomForest(num_trees).train(
            features=training_set,
            classProperty='loss',
            inputProperties=training_stack.bandNames()
        )

        # Aplicar el modelo a la imagen completa
        raw_risk_map = training_stack.classify(classifier)

        # Evaluar el modelo con el conjunto de prueba
        testing_classified = testing_set.classify(classifier)
        predictions = testing_classified.aggregate_array("classification").getInfo()
        true_values = testing_classified.aggregate_array("loss").getInfo()

        if len(predictions) > 0 and len(true_values) > 0:
            r2 = r2_score(true_values, predictions)
            print(f"Coeficiente de determinación R²: {r2:.4f}")
        else:
            print("Advertencia: No se pudieron calcular métricas de evaluación.")

        # Normalizar el riesgo entre 0 y 1
        stats = raw_risk_map.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=aoi.geometry(),
            scale=1000,  # Aumentar la escala para reducir el número de píxeles
            maxPixels=1e13  # Aumentar el límite de píxeles
        ).getInfo()
        min_val = stats.get("classification_min", 0)
        max_val = stats.get("classification_max", 1)
        risk_map = raw_risk_map.subtract(min_val).divide(max_val - min_val)

        # Exportar resultado
        export_task = ee.batch.Export.image.toDrive(
            image=risk_map,
            description='deforestation_risk_map',
            folder='GEE_Exports',
            fileNamePrefix='deforestation_risk',
            scale=export_resolution,
            region=aoi.geometry().bounds(),
            fileFormat='GeoTIFF'
        )
        export_task.start()
        print("Exportación iniciada correctamente.")

        # Visualización en mapa
        Map = geemap.Map()
        Map.centerObject(aoi, 6)
        Map.addLayer(forest_loss, {'min': 0, 'max': 1, 'palette': ['black', 'red']}, "Forest Loss")
        Map.addLayer(points, {}, "Sampled Points")
        Map.addLayer(risk_map, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, "Riesgo de Deforestación")
        return Map

    except Exception as e:
        print(f"Error: {e}")
        return None

# Ejemplo de uso:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM1_NAME", "Amazonas"))
Map = calculate_deforestation_risk(aoi, export_resolution=20)
if Map:
    Map

Número de puntos de muestreo generados: 9775
Coeficiente de determinación R²: -0.0243
Exportación iniciada correctamente.
